In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy

In [ ]:
# --- 3D ConvNeXt-like Components ---

class LayerNorm3D(nn.Module):
    """ LayerNorm that supports 3D input (B, C, D, H, W). """
    def __init__(self, normalized_shape, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        # PyTorch's LayerNorm applies over the last dimensions.
        # For (B, C, D, H, W), we want it over (D, H, W) or (C, D, H, W).
        # ConvNeXt uses it over channels after depthwise conv, then over features.
        # Let's apply it over (D, H, W) which is typical for spatial norms.
        if isinstance(normalized_shape, int):
             # Assume channel dimension if int
             self.normalized_shape = (normalized_shape,)
        else:
             self.normalized_shape = tuple(normalized_shape) # Should be (C, D, H, W) or (D, H, W)

    def forward(self, x):
        # Assuming x is (B, C, D, H, W)
        # Apply norm over the last self.normalized_shape dimensions
        # This requires permuting for LayerNorm and then permuting back
        # A simpler way is to compute mean/var manually or use F.layer_norm
        
        # F.layer_norm expects (..., normalized_shape)
        # We have (B, C, D, H, W) and want to normalize (D, H, W) or (C, D, H, W)
        
        # Normalize over spatial dimensions (D, H, W) - similar to InstanceNorm but batch-wise mean/var
        # Alternative: Normalize over (C, D, H, W) - less common but matches some interpretations
        
        # Let's try normalizing over (D, H, W)
        mean = x.mean(dim=(-3, -2, -1), keepdim=True)
        var = x.var(dim=(-3, -2, -1), keepdim=True, unbiased=False)
        x = (x - mean) / torch.sqrt(var + self.eps)
        
        # Reshape weight and bias to match x dims for broadcasting
        # weight and bias are (C,) if normalized_shape is C
        # need to reshape to (1, C, 1, 1, 1)
        if len(self.normalized_shape) == 1 and self.normalized_shape[0] == x.size(1):
             weight = self.weight.view(1, -1, 1, 1, 1)
             bias = self.bias.view(1, -1, 1, 1, 1)
        else:
             # This case is complex if normalized_shape is (D, H, W) - need to match
             # For this example, let's assume channel-wise normalization after depthwise
             # This matches ConvNeXt paper's application slightly better (applied to channels)
             # Let's redefine to normalize over channels: (B, C, D, H, W) -> norm over C
             mean = x.mean(dim=1, keepdim=True)
             var = x.var(dim=1, keepdim=True, unbiased=False)
             x = (x - mean) / torch.sqrt(var + self.eps)
             weight = self.weight.view(1, -1, 1, 1, 1) # Assuming normalized_shape is C
             bias = self.bias.view(1, -1, 1, 1, 1)


        return x * weight + bias

class ConvNeXtBlock3D(nn.Module):
    """ ConvNeXt Block adapted for 3D input. """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        # Larger depthwise kernel (e.g., 7x7x7)
        self.dwconv = nn.Conv3d(dim, dim, kernel_size=7, padding=3, groups=dim)
        # LayerNorm applied over channels
        self.norm = LayerNorm3D(dim)
        # 1x1x1 convolution to expand channels (factor 4)
        self.pwconv1 = nn.Linear(dim, 4 * dim)
        # GELU activation
        self.act = nn.GELU()
        # 1x1x1 convolution to contract channels
        self.pwconv2 = nn.Linear(4 * dim, dim)
        
        # Layer scale
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                   requires_grad=True) if layer_scale_init_value > 0 else None
        
        # Drop path (stochastic depth) - not implemented in this basic example,
        # but can be added for regularization.

    def forward(self, x):
        # x is (B, C, D, H, W)
        input = x
        
        # Depthwise convolution
        x = self.dwconv(x)
        
        # Permute for LayerNorm and Linear layers: (B, C, D, H, W) -> (B, D, H, W, C)
        # Apply norm over channels: (B, C, D, H, W) -> norm over C
        # Let's stick to channel-wise norm after DW conv as per ConvNeXt
        x = self.norm(x) # Norm is over channels (C)

        # Permute back for Linear: (B, C, D, H, W) -> (B, D, H, W, C)
        x = x.permute(0, 2, 3, 4, 1) 
        
        # Pointwise convolutions (implemented as Linear layers)
        x = self.pwconv1(x) # (B, D, H, W, C*4)
        x = self.act(x)
        x = self.pwconv2(x) # (B, D, H, W, C)
        
        # Apply layer scale if gamma is used
        if self.gamma is not None:
            x = self.gamma * x # (B, D, H, W, C) * (C,) broadcasts
        
        # Permute back to (B, C, D, H, W) for residual connection
        x = x.permute(0, 4, 1, 2, 3)
        
        # Residual connection
        x = input + x

        return x

In [ ]:
# --- Transformer Components ---

class PositionalEncoding(nn.Module):
    """ Basic fixed sinusoidal positional encoding. """
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # Add batch dimension
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x is (B, SeqLen, embed_dim)
        # Add positional encoding to the tokens
        return x + self.pe[:, :x.size(1)]

In [ ]:
# --- Full Model Combining Components ---

class WaveformInversionModel(nn.Module):
    def __init__(self, 
                 input_dims=(1, 128, 512, 128), # (C, S, T, R) - Example: 1 channel, 128 sources, 512 time steps, 128 receivers
                 output_dims=(1, 256, 256),   # (C, H, W) - Example: 1 channel velocity map, 256x256 spatial
                 convnext_channels=[32, 64, 128, 256], # Channels for 3D CNN stages
                 convnext_depths=[2, 2, 6, 2],      # Number of blocks in each 3D CNN stage
                 convnext_downsample_strides=[(1, 2, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2)], # Stride for downsampling between stages
                 transformer_layers=4, 
                 transformer_heads=8, 
                 transformer_embed_dim=256, # Transformer embedding dimension
                 transformer_ffn_dim=1024,
                 decoder_channels=[128, 64, 32], # Channels for 2D decoder stages
                 decoder_upsample_strides=[(2, 2), (2, 2), (2, 2)], # Stride for upsampling (H, W)
                 ):
        super().__init__()
        
        in_channels, in_S, in_T, in_R = input_dims
        out_channels, out_H, out_W = output_dims

        self.convnext_channels = convnext_channels
        self.convnext_depths = convnext_depths
        self.convnext_downsample_strides = convnext_downsample_strides
        self.transformer_embed_dim = transformer_embed_dim
        self.decoder_upsample_strides = decoder_upsample_strides

        # --- 3D ConvNeXt Encoder ---
        stages = []
        # Initial convolution
        stages.append(nn.Conv3d(in_channels, convnext_channels[0], kernel_size=4, stride=convnext_downsample_strides[0], padding=1))
        # Calculate dimensions after initial conv
        current_S = math.floor((in_S - 4 + 2*1) / convnext_downsample_strides[0][0]) + 1
        current_T = math.floor((in_T - 4 + 2*1) / convnext_downsample_strides[0][1]) + 1
        current_R = math.floor((in_R - 4 + 2*1) / convnext_downsample_strides[0][2]) + 1

        for i in range(len(convnext_channels)):
            dim = convnext_channels[i]
            # Add ConvNeXt blocks for the stage
            stage_blocks = [
                ConvNeXtBlock3D(dim) for _ in range(convnext_depths[i])
            ]
            stages.append(nn.Sequential(*stage_blocks))

            # Add downsampling layer between stages (except after the last stage)
            if i < len(convnext_channels) - 1:
                next_dim = convnext_channels[i+1]
                downsample_stride = convnext_downsample_strides[i+1]
                stages.append(nn.Sequential(
                    LayerNorm3D(dim), # Norm before downsampling
                    nn.Conv3d(dim, next_dim, kernel_size=2, stride=downsample_stride) # 2x2x2 conv with stride
                ))
                current_S = math.floor((current_S - 2) / downsample_stride[0]) + 1
                current_T = math.floor((current_T - 2) / downsample_stride[1]) + 1
                current_R = math.floor((current_R - 2) / downsample_stride[2]) + 1
        
        self.encoder_3d = nn.Sequential(*stages)
        
        self.final_3d_channels = convnext_channels[-1]
        self.final_S = current_S
        self.final_T = current_T
        self.final_R = current_R
        self.sequence_length = self.final_S * self.final_T * self.final_R

        print(f"3D Encoder output shape (before flatten): (B, {self.final_3d_channels}, {self.final_S}, {self.final_T}, {self.final_R})")
        print(f"Transformer sequence length: {self.sequence_length}")

        # --- Project to Transformer Embedding Dim ---
        self.encoder_to_transformer_proj = nn.Linear(self.final_3d_channels, transformer_embed_dim)
        self.pos_embedding = PositionalEncoding(transformer_embed_dim, max_len=self.sequence_length)

        # --- Transformer Encoder ---
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=transformer_embed_dim,
            nhead=transformer_heads,
            dim_feedforward=transformer_ffn_dim,
            batch_first=True # Input expects (batch, seq, dim)
        )
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=transformer_layers)

        # --- Map Transformer Output to 2D Latent Grid ---
        # We need to reshape the sequence (B, SeqLen, embed_dim) to (B, C_dec_init, H_latent, W_latent)
        # Where H_latent * W_latent = SeqLen.
        # Let's try to find suitable H_latent, W_latent from SeqLen = S'*T'*R'
        # This mapping is arbitrary without specific domain knowledge or learned approach.
        # For demonstration, let's try to make H_latent, W_latent roughly spatial dimensions.
        # S' relates to sources, R' relates to receivers (both spatial). T' relates to time.
        # Let's try to map S'xR' to H_latent x W_latent and somehow incorporate T'.
        # A simpler way is to calculate factors of SeqLen and choose H_latent, W_latent.
        # Example: SeqLen = 8192 = 64 * 128. Let H_latent = 64, W_latent = 128.
        
        # Need to calculate the total upsampling factor needed in 2D decoder
        total_upsample_H = out_H / H_latent # This requires H_latent to be a divisor of out_H
        total_upsample_W = out_W / W_latent # This requires W_latent to be a divisor of out_W
        
        # Let's assume we can find H_latent, W_latent such that SeqLen = H_latent * W_latent
        # and total_upsample_H, total_upsample_W are powers of 2 or match decoder strides.
        
        # For now, let's make a simplifying assumption:
        # We will infer H_latent, W_latent such that H_latent * W_latent = SeqLen
        # and the total upsampling factors match the target output dimensions.
        # This requires careful design of convnext_downsample_strides and decoder_upsample_strides
        # and finding suitable H_latent, W_latent.
        
        # A safer approach for a demo is to calculate SeqLen, choose *any* suitable H_latent, W_latent
        # such that H_latent * W_latent = SeqLen, and *then* define decoder strides to reach the target H, W.
        
        # Let's calculate the target H_latent, W_latent based on the final encoder dims:
        # Try to roughly map S' to H_latent, R' to W_latent, and somehow collapse T'.
        # This is tricky. Let's just find factors of SeqLen.
        # Example: SeqLen = 8192. Factors include (64, 128). Let's try H_latent=64, W_latent=128.
        
        # --- Determining H_latent, W_latent ---
        # This is a design choice. H_latent * W_latent must equal self.sequence_length.
        # The decoder will upsample from (H_latent, W_latent) to (out_H, out_W).
        # The product of decoder upsample strides must be out_H/H_latent and out_W/W_latent.
        
        # Let's make a pragmatic choice for H_latent, W_latent for the example:
        # Find a pair of factors (h, w) for SeqLen such that h and w are reasonably close
        # and h/out_H, w/out_W allow integer total upsampling factors.
        
        # Simplified approach: Find factors. Just pick one combination for the demo.
        # Let's assume H_latent and W_latent are determined based on the structure, e.g.,
        # maybe collapse T' dimension and roughly scale S' and R' to H_latent and W_latent.
        # Let's assume for this example that H_latent = self.final_S * factor_S, W_latent = self.final_R * factor_R
        # and T' dimension is somehow incorporated into channels or collapsed differently.
        # This requires the total size self.final_S * self.final_T * self.final_R to be reshaped into H_latent * W_latent.
        # This implies H_latent * W_latent must equal self.sequence_length.

        # A common approach: Assume H_latent, W_latent are derived from the input dimensions by total stride.
        # E.g., if total downsampling is 8x in S, 16x in T, 8x in R.
        # S'=S/8, T'=T/16, R'=R/8. SeqLen = (S/8)*(T/16)*(R/8).
        # If output is HxW, maybe H_latent = H / total_decoder_stride_H, W_latent = W / total_decoder_stride_W.
        # And we need H_latent * W_latent = SeqLen.
        
        # Let's define H_latent and W_latent first based on required upsampling.
        # Decoder has N stages with strides s_h_i, s_w_i. Total stride H = prod(s_h_i), Total stride W = prod(s_w_i).
        # H_latent = out_H // Total_stride_H, W_latent = out_W // Total_stride_W.
        # This requires SeqLen = H_latent * W_latent.

        total_upsample_H = math.prod([s[0] for s in decoder_upsample_strides])
        total_upsample_W = math.prod([s[1] for s in decoder_upsample_strides])
        
        self.H_latent = out_H // total_upsample_H
        self.W_latent = out_W // total_upsample_W

        print(f"Decoder latent shape: ({self.H_latent}, {self.W_latent})")
        print(f"Required sequence length for reshape: {self.H_latent * self.W_latent}")
        
        # Ensure the calculated latent size matches the transformer sequence length
        # This might require adjusting ConvNeXt strides or decoder strides
        if self.H_latent * self.W_latent != self.sequence_length:
             raise ValueError(f"Mismatch between transformer sequence length ({self.sequence_length}) "
                             f"and required latent decoder size ({self.H_latent * self.W_latent}). "
                             f"Adjust ConvNeXt strides or decoder strides.")

        self.transformer_to_decoder_proj = nn.Linear(transformer_embed_dim, decoder_channels[0] * self.H_latent * self.W_latent)
        # Alternatively, project seq items to channels and reshape
        self.transformer_output_channels = decoder_channels[0]
        self.transformer_to_decoder_reshape_proj = nn.Linear(transformer_embed_dim, self.transformer_output_channels)


        # --- 2D Decoder ---
        decoder_stages = []
        in_c = self.transformer_output_channels # Starting channels for 2D decoder
        current_H, current_W = self.H_latent, self.W_latent

        for i in range(len(decoder_channels)):
            out_c = decoder_channels[i]
            upsample_stride = decoder_upsample_strides[i]
            
            # Use ConvTranspose2d for upsampling
            decoder_stages.append(nn.ConvTranspose2d(in_c, out_c, kernel_size=upsample_stride, stride=upsample_stride))
            
            current_H *= upsample_stride[0]
            current_W *= upsample_stride[1]
            in_c = out_c

        self.decoder_2d = nn.Sequential(*decoder_stages)

        # Final output layer
        self.final_conv = nn.Conv2d(decoder_channels[-1], out_channels, kernel_size=3, padding=1) # Output velocity channel(s)

    def forward(self, x):
        # Input x is (B, S, T, R)
        
        # Add channel dimension for 3D Conv: (B, 1, S, T, R)
        x = x.unsqueeze(1)

        # 3D ConvNeXt Encoder
        x = self.encoder_3d(x) # (B, C_3d, S', T', R')

        # Flatten and Project to Transformer
        B, C, S_prime, T_prime, R_prime = x.shape
        x = x.permute(0, 2, 3, 4, 1) # (B, S', T', R', C_3d)
        x = x.reshape(B, -1, C)      # (B, SeqLen, C_3d) where SeqLen = S'*T'*R'

        # Project channels to transformer_embed_dim
        x = self.encoder_to_transformer_proj(x) # (B, SeqLen, embed_dim)

        # Add Positional Encoding
        x = self.pos_embedding(x) # (B, SeqLen, embed_dim)

        # Transformer Encoder
        x = self.transformer_encoder(x) # (B, SeqLen, embed_dim)

        # Map to 2D Latent Grid for Decoder
        # Project embed_dim to transformer_output_channels (which is decoder_channels[0])
        x = self.transformer_to_decoder_reshape_proj(x) # (B, SeqLen, C_dec_init)

        # Reshape sequence to latent 2D grid: (B, C_dec_init, H_latent, W_latent)
        # Need to permute first to get channels first: (B, C_dec_init, SeqLen)
        x = x.permute(0, 2, 1) # (B, C_dec_init, SeqLen)
        
        # Reshape SeqLen into H_latent * W_latent
        x = x.reshape(B, self.transformer_output_channels, self.H_latent, self.W_latent) # (B, C_dec_init, H_latent, W_latent)

        # 2D Decoder
        x = self.decoder_2d(x) # (B, C_final, H, W)

        # Final Output Layer
        x = self.final_conv(x) # (B, 1, H, W)

        return x

In [ ]:
# --- EMA Utility ---

class EMA:
    """ 
    Exponential Moving Average
    Maintains a shadow copy of the model weights and updates them
    using a decay rate during training.
    """
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

        # Register model parameters
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        """ Update the shadow weights. """
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        """ Copy shadow weights to the model parameters. """
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data.clone() # Store original weights
                param.data.copy_(self.shadow[name])

    def restore(self):
        """ Restore original model weights from backup. """
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data.copy_(self.backup[name])
        self.backup = {} # Clear backup after restoring


In [ ]:
# --- Example Usage ---

if __name__ == "__main__":
    # Define input and output dimensions
    # (C, S, T, R) where S=sources, T=time, R=receivers
    input_dims = (1, 64, 256, 64) # Example: 1 channel, 64 sources, 256 time, 64 receivers
    # (C, H, W) where H=height, W=width of velocity map
    output_dims = (1, 128, 128) # Example: 1 channel, 128x128 velocity map

    # Configure model parameters
    convnext_channels = [32, 64, 128]
    convnext_depths = [2, 2, 2]
    # Example strides: Initial conv (1,2,1), then stages downsample by (2,2,2), (2,2,2)
    convnext_downsample_strides = [(1, 2, 1), (2, 2, 2), (2, 2, 2)]
    
    transformer_layers = 3
    transformer_heads = 4
    transformer_embed_dim = 128
    transformer_ffn_dim = 512

    decoder_channels = [64, 32]
    # Example strides: Need to go from latent (H_latent, W_latent) to output (128, 128)
    # If latent is (16, 16), need 8x upsampling. Strides (2,2), (2,2), (2,2) give 8x.
    # Let's calculate required H_latent, W_latent
    total_upsample_H = 1
    total_upsample_W = 1
    for s_h, s_w in [(2,2), (2,2)]: # Example decoder strides
        total_upsample_H *= s_h
        total_upsample_W *= s_w
    
    # Required latent size calculation based on target output and *proposed* decoder strides
    H_latent_req = output_dims[1] // total_upsample_H
    W_latent_req = output_dims[2] // total_upsample_W
    
    print(f"Required latent decoder size based on decoder strides ({[(2,2), (2,2)]}): ({H_latent_req}, {W_latent_req})")
    
    decoder_upsample_strides = [(2, 2), (2, 2)] # Example 2D decoder upsampling strides

    # --- Check if dimensions match ---
    # Calculate final 3D encoder output dimensions given input_dims and convnext_downsample_strides
    c, s, t, r = input_dims
    
    # Initial conv
    s = math.floor((s - 4 + 2*1) / convnext_downsample_strides[0][0]) + 1
    t = math.floor((t - 4 + 2*1) / convnext_downsample_strides[0][1]) + 1
    r = math.floor((r - 4 + 2*1) / convnext_downsample_strides[0][2]) + 1
    
    # Stages downsampling
    for i in range(1, len(convnext_downsample_strides)):
        ds_stride = convnext_downsample_strides[i]
        s = math.floor((s - 2) / ds_stride[0]) + 1
        t = math.floor((t - 2) / ds_stride[1]) + 1
        r = math.floor((r - 2) / ds_stride[2]) + 1
        
    final_S = s
    final_T = t
    final_R = r
    calculated_seq_len = final_S * final_T * final_R
    
    print(f"Calculated 3D encoder output spatial dims: ({final_S}, {final_T}, {final_R})")
    print(f"Calculated transformer sequence length: {calculated_seq_len}")

    # Check if calculated SeqLen matches required latent size H_latent_req * W_latent_req
    if calculated_seq_len != H_latent_req * W_latent_req:
         print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
         print("!! WARNING: Calculated sequence length from 3D encoder does NOT match !!")
         print("!! required latent decoder size based on decoder strides.   !!")
         print("!! This model configuration will fail due to reshape error. !!")
         print(f"!! SeqLen ({calculated_seq_len}) != H_latent*W_latent ({H_latent_req * W_latent_req}) !!")
         print("!! Adjust convnext_downsample_strides or decoder_upsample_strides to match !!")
         print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
         
         # Let's adjust decoder strides to make the example runnable.
         # We need total_upsample_H * total_upsample_W such that
         # (output_H / total_upsample_H) * (output_W / total_upsample_W) == calculated_seq_len
         # This is complex. Let's simplify the example config to make it work.
         # Target H, W = 128, 128
         # Let's aim for latent (16, 16). SeqLen = 256.
         # Need total upsample 8x in H, 8x in W. Decoder strides: (2,2), (2,2), (2,2).
         decoder_upsample_strides = [(2,2), (2,2), (2,2)]
         decoder_channels = [64, 32, 16] # Needs one more stage
         
         # Recalculate required latent size
         total_upsample_H = math.prod([s[0] for s in decoder_upsample_strides])
         total_upsample_W = math.prod([s[1] for s in decoder_upsample_strides])
         H_latent_req = output_dims[1] // total_upsample_H # 128 // 8 = 16
         W_latent_req = output_dims[2] // total_upsample_W # 128 // 8 = 16
         required_seq_len = H_latent_req * W_latent_req # 16 * 16 = 256
         print(f"Adjusted decoder strides and required latent size: ({H_latent_req}, {W_latent_req}), SeqLen={required_seq_len}")

         # Now adjust ConvNeXt strides so calculated_seq_len = required_seq_len = 256
         # Input (64, 256, 64)
         # Need final_S * final_T * final_R = 256
         # Let's try downsampling: S/8, T/16, R/8.
         # (64/8) * (256/16) * (64/8) = 8 * 16 * 8 = 1024. Too big.
         # Need more aggressive downsampling or different structure.
         # Maybe S/8, T/32, R/8? (64/8)*(256/32)*(64/8) = 8*8*8 = 512. Still too big.
         # Maybe S/16, T/16, R/16? (64/16)*(256/16)*(64/16) = 4*16*4 = 256. This works!
         
         convnext_downsample_strides = [(2, 2, 2), (2, 2, 2), (2, 2, 2)] # Total 8x downsampling initially
         # Recalculate dimensions with new strides:
         s_new, t_new, r_new = input_dims[1:]
         s_new = math.floor((s_new - 4 + 2*1) / convnext_downsample_strides[0][0]) + 1 # (64-4+2)/2 + 1 = 31+1 = 32?
         t_new = math.floor((t_new - 4 + 2*1) / convnext_downsample_strides[0][1]) + 1 # (256-4+2)/2 + 1 = 127+1 = 128?
         r_new = math.floor((r_new - 4 + 2*1) / convnext_downsample_strides[0][2]) + 1 # (64-4+2)/2 + 1 = 31+1 = 32?
         # Using kernel 4, stride 2 initially.
         
         # Let's use kernel 2, stride 2 between stages after the first block as planned.
         # Input (64, 256, 64)
         # Stage 1 (initial conv, kernel 4, stride 2): (31, 127, 31) using floor. Use ceil for simplicity in design? No, floor is correct.
         # The calculation needs to be precise based on padding.
         # Let's adjust input dims slightly or make strides simpler.
         # Simpler strides: Initial (1,1,1) kernel 7, padding 3 -> (64, 256, 64)
         # Then stages downsample 2x, 2x, 2x
         # (64, 256, 64) -> (32, 128, 32) -> (16, 64, 16) -> (8, 32, 8)
         # SeqLen = 8 * 32 * 8 = 2048. This doesn't match 256.
         
         # Let's use the required SeqLen (256) and H_latent, W_latent (16, 16) as fixed points
         # and work backwards to find suitable encoder downsampling.
         # Final encoder dims (S', T', R') must satisfy S'*T'*R' = 256.
         # Possible (S', T', R'): (8, 4, 8), (4, 8, 8), (8, 8, 4), (16, 4, 4), etc.
         # If input is (64, 256, 64), need total downsampling of 64/S', 256/T', 64/R'.
         # E.g., (8, 4, 8) -> Need 8x in S, 64x in T, 8x in R.
         # Can we achieve 64x in T with ConvNeXt strides like (1,2,1), (2,2,2), (2,2,2)?
         # Initial (1,2,1) -> 2x in T. Then three (2,2,2) stages -> 2*2*2 = 8x in T. Total 2 * 8 = 16x in T. Not 64x.
         
         # The dimension matching is crucial and depends entirely on the specific layer configurations.
         # Let's define a config that *should* work for the (64, 256, 64) input and (128, 128) output.
         # Target latent (16, 16), SeqLen 256. Upsample needed 8x in H, 8x in W. Decoder strides (2,2), (2,2), (2,2) (3 stages).
         decoder_upsample_strides = [(2, 2), (2, 2), (2, 2)]
         decoder_channels = [64, 32, 16] # 3 stages
         # Needs input to decoder be (B, 64, 16, 16)
         
         # Need S'*T'*R' = 256 from encoder.
         # Let's use input (64, 128, 64) to simplify. (Different input dims)
         input_dims = (1, 64, 128, 64)
         # Initial stride (1,1,1) k=4, p=1 -> (61, 125, 61) - complex.
         # Simpler: initial stride (1,1,1), k=7, p=3 -> (64, 128, 64)
         # Then 3 stages, each downsample 2x, 2x, 2x.
         # (64, 128, 64) -> (32, 64, 32) -> (16, 32, 16) -> (8, 16, 8)
         # SeqLen = 8 * 16 * 8 = 1024.
         # Need SeqLen = 256. How about (4, 8, 8)? 4*8*8=256.
         # From (64, 128, 64) to (4, 8, 8). Total downsampling: S=16x, T=16x, R=8x.
         # Initial (1,1,1). Stage 1 (2,2,1), Stage 2 (2,2,2), Stage 3 (4,4,4)?
         # Let's try simpler strides:
         # Initial (1,1,1) k=7, p=3
         # Stage 1 (4,4,2) -> (16, 32, 32)
         # Stage 2 (4,4,4) -> (4, 8, 8) -> SeqLen = 4*8*8=256. This works!
         
         convnext_channels = [64, 128, 256] # 3 stages
         convnext_depths = [2, 2, 2]
         convnext_downsample_strides = [(1, 1, 1), (4, 4, 2), (4, 4, 4)] # Strides *between* stages (plus initial k=7, p=3).
         # Initial conv should have kernel and stride to transition from input_dims channels to convnext_channels[0]
         # and *optionally* downsample. Let's make the initial conv just change channels, stride 1.
         # And the *first* downsampling layer is after the first stage's blocks.
         
         print("\n--- Adjusted Configuration for Dimensional Consistency ---")
         input_dims = (1, 64, 128, 64) # New input dims
         output_dims = (1, 128, 128)   # Target output dims
         
         # ConvNeXt Encoder Config
         # Initial conv: input_dims[0] -> convnext_channels[0], stride 1
         # Stages: convnext_depths[i] blocks
         # Downsampling layers *between* stages i and i+1.
         convnext_channels = [32, 64, 128] # 3 stages
         convnext_depths = [2, 2, 2]      # 2 blocks per stage
         # Downsampling strides between stages:
         # From stage 0 -> stage 1: (4, 4, 2)
         # From stage 1 -> stage 2: (4, 4, 4)
         convnext_inter_stage_downsample_strides = [(4, 4, 2), (4, 4, 4)] # List length = num_stages - 1
         
         # Decoder Config
         # Target output (128, 128)
         # Target latent (16, 16), SeqLen 256. Upsample 8x, 8x.
         decoder_upsample_strides = [(2, 2), (2, 2), (2, 2)] # 3 stages
         decoder_channels = [64, 32, 16] # Number of stages should match upsample strides length + 1 (or length)

         # Let's verify the pipeline with this config:
         # Input: (B, 1, 64, 128, 64)
         # Initial Conv (1->32, k=3, p=1, s=1): (B, 32, 64, 128, 64) # Using k=3, p=1 for simplicity
         # Stage 0 blocks (2 blocks): (B, 32, 64, 128, 64)
         # Downsample 0->1 (32->64, k=2, s=(4,4,2)): (B, 64, (64-2)/4+1, (128-2)/4+1, (64-2)/2+1) = (B, 64, 16, 32, 32) - Using floor
         # Stage 1 blocks (2 blocks): (B, 64, 16, 32, 32)
         # Downsample 1->2 (64->128, k=2, s=(4,4,4)): (B, 128, (16-2)/4+1, (32-2)/4+1, (32-2)/4+1) = (B, 128, 4, 8, 8)
         # Stage 2 blocks (2 blocks): (B, 128, 4, 8, 8)
         # Final Encoder dims: S'=4, T'=8, R'=8. C_3d = 128.
         # SeqLen = 4 * 8 * 8 = 256. Matches required 256! Latent (16, 16).
         # Transformer embed_dim = 128 (let's match final_3d_channels for simplicity, or keep 256)
         transformer_embed_dim = 128
         transformer_ffn_dim = 4 * transformer_embed_dim # Standard
         
         # Transformer input: (B, 256, 128)
         # Transformer output: (B, 256, 128)
         # Project to decoder initial channels: (B, 256, 64)
         # Reshape to (B, 64, 16, 16). This works.
         # Decoder stages:
         # (B, 64, 16, 16) -> ConvT(64->32, k=2, s=2) -> (B, 32, 32, 32) # Using k=s here for simplicity
         # (B, 32, 32, 32) -> ConvT(32->16, k=2, s=2) -> (B, 16, 64, 64)
         # (B, 16, 64, 64) -> ConvT(16->16, k=2, s=2) -> (B, 16, 128, 128) # Final decoder stage output channels

         decoder_channels = [32, 16, 16] # Channels *after* each ConvTranspose stage
         # Initial channels for 2D decoder is convnext_channels[-1] (128) -> projected?
         # Let's set the initial channels for the *2D decoder* separately.
         decoder_2d_start_channels = 64 # Should be <= transformer_embed_dim after proj

         print("Using adjusted configuration:")
         print(f"  Input Dims: {input_dims}")
         print(f"  Output Dims: {output_dims}")
         print(f"  ConvNeXt Channels: {convnext_channels}")
         print(f"  ConvNeXt Depths: {convnext_depths}")
         # Print initial conv config assumed for calculation
         print(f"  ConvNeXt Initial Conv: k=3, p=1, s=1 (adjust if needed)")
         print(f"  ConvNeXt Inter-Stage Downsample Strides: {convnext_inter_stage_downsample_strides}")
         print(f"  Calculated Final Encoder Spatial Dims: ({final_S}, {final_T}, {final_R})")
         print(f"  Calculated Transformer SeqLen: {calculated_seq_len}")
         print(f"  Transformer Embed Dim: {transformer_embed_dim}")
         print(f"  Decoder 2D Start Channels (after Transformer proj): {decoder_2d_start_channels}")
         print(f"  Decoder Latent Size (H_latent, W_latent): ({H_latent_req}, {W_latent_req})")
         print(f"  Decoder Upsample Strides (H, W): {decoder_upsample_strides}")
         print(f"  Decoder Output Channels (per stage): {decoder_channels}")
         print("-" * 30)

         # Update model initialization parameters based on adjusted config
         model = WaveformInversionModel(
             input_dims=input_dims,
             output_dims=output_dims,
             convnext_channels=convnext_channels,
             convnext_depths=convnext_depths,
             # Need to pass the structure to the model
             # A more flexible model class would take a list of stage configs
             # For this demo, hardcode the initial conv and inter-stage downsampling
             transformer_layers=transformer_layers,
             transformer_heads=transformer_heads,
             transformer_embed_dim=transformer_embed_dim,
             transformer_ffn_dim=transformer_ffn_dim,
             decoder_channels=decoder_channels, # These are the *output* channels of ConvTranspose layers
             decoder_upsample_strides=decoder_upsample_strides,
             # Add parameters to pass calculated H_latent, W_latent and SeqLen
             _H_latent=H_latent_req,
             _W_latent=W_latent_req,
             _sequence_length=calculated_seq_len,
             _decoder_2d_start_channels=decoder_2d_start_channels
         )
         
    else:
        # Use original configuration if it happened to match (unlikely with random choices)
         model = WaveformInversionModel(
             input_dims=input_dims,
             output_dims=output_dims,
             convnext_channels=convnext_channels,
             convnext_depths=convnext_depths,
             convnext_downsample_strides=convnext_downsample_strides,
             transformer_layers=transformer_layers,
             transformer_heads=transformer_heads,
             transformer_embed_dim=transformer_embed_dim,
             transformer_ffn_dim=transformer_ffn_dim,
             decoder_channels=decoder_channels,
             decoder_upsample_strides=decoder_upsample_strides,
             # Add parameters to pass calculated H_latent, W_latent and SeqLen
             _H_latent=H_latent_req,
             _W_latent=W_latent_req,
             _sequence_length=calculated_seq_len,
             _decoder_2d_start_channels=decoder_channels[0] # Use first decoder channel as start
         )
         print("\nUsing original (likely failing) configuration.") # Will print error inside __init__

    print("\nModel Architecture:")
    # print(model) # Can be very verbose

    # Create dummy data
    # (batch_size, num_sources, time_steps, num_receivers)
    batch_size = 2
    dummy_input = torch.randn(batch_size, input_dims[1], input_dims[2], input_dims[3])
    print(f"\nDummy Input Shape: {dummy_input.shape}")

    # Instantiate EMA
    ema_decay = 0.999
    ema = EMA(model, ema_decay)
    print(f"EMA initialized with decay: {ema_decay}")

    # Example Forward Pass
    with torch.no_grad(): # Typically inference/evaluation uses no_grad
        # Optionally apply EMA shadow weights for evaluation
        # ema.apply_shadow() 
        
        output = model(dummy_input)
        
        # Restore original weights after evaluation if EMA was applied
        # ema.restore() 

    print(f"Output Shape: {output.shape}")
    # Expected output shape: (batch_size, 1, output_H, output_W)
    expected_output_shape = (batch_size, output_dims[0], output_dims[1], output_dims[2])
    assert output.shape == expected_output_shape
    print("Output shape matches expected shape.")

    # Example of EMA update (in a training loop)
    # Assume you have loss, optimizer, etc.
    # loss.backward()
    # optimizer.step()
    # ema.update() # <-- Call this after optimizer.step()

### How to Use in Training:

1. Instantiate the model.
2. Instantiate the optimizer.
3. Instantiate the EMA utility: ema = EMA(model, decay=0.999).
4. In your training loop, after optimizer.step() (and zeroing gradients), call ema.update().
5. For evaluation or inference:
 - Call ema.apply_shadow() to load the averaged weights.
 - Run your evaluation/inference pass: with torch.no_grad(): output = model(input).
 - Call ema.restore() to load the original training weights back if you plan to continue training.

### Explanation:

1. **LayerNorm3D:** A simple implementation of Layer Normalization for 3D tensors. Standard nn.LayerNorm works on the last normalized_shape dimensions. We adapt it to normalize over the channel dimension (C) for (B, C, D, H, W) input, similar to how LayerNorm is often used in ConvNeXt after depthwise convolutions.
2. **ConvNeXtBlock3D:** Implements the core ConvNeXt block structure adapted to 3D:
 - Large kernel depthwise 3D convolution (nn.Conv3d with groups=dim).
 - LayerNorm3D over channels.
 - Pointwise convolutions (nn.Linear after permuting dimensions) with GELU activation, forming an inverted bottleneck structure (expand channels, then contract).
 - Optional Layer Scale (gamma).
 - Residual connection.
3. **PositionalEncoding:** A standard sinusoidal positional encoding module for adding positional information to the sequence of tokens fed into the Transformer.
4. **WaveformInversionModel:**
 - 3D Encoder: A nn.Sequential stack. It starts with an initial 3D convolution to adjust channel count and potentially downsample. Then, it iterates through defined stages, adding ConvNeXtBlock3Ds followed by a downsampling layer (nn.Conv3d with stride > 1) between stages. The strides and kernel sizes are critical for determining the final 3D spatial dimensions (final_S, final_T, final_R).
 - Flatten & Project: The output (B, C_3d, S', T', R') is permuted and reshaped into a sequence (B, SeqLen, C_3d). A Linear layer projects the channels C_3d to the required transformer_embed_dim. Positional encoding is added to this sequence.
 - Transformer Encoder: A standard nn.TransformerEncoder consisting of multiple TransformerEncoderLayers processes the sequence.
 - Map to 2D Latent: The output sequence (B, SeqLen, embed_dim) is first projected to decoder_2d_start_channels using a Linear layer. Then, it's permuted and reshaped from (B, decoder_2d_start_channels, SeqLen) into (B, decoder_2d_start_channels, H_latent, W_latent). This reshape requires H_latent * W_latent == SeqLen. The __init__ method includes a check and raises an error if the dimensions don't match, as getting this right with arbitrary inputs/outputs/strides requires careful planning or a more complex dynamic reshaping/projection mechanism. The example config provided after the warning is designed to meet this requirement for the specific input/output sizes.
 - 2D Decoder: A nn.Sequential stack of nn.ConvTranspose2d layers. These layers upsample the 2D grid from (H_latent, W_latent) to the target output size (H, W). The strides determine the upsampling factor at each stage.
 - Output Layer: A final nn.Conv2d maps the decoder's output channels to the single channel required for the velocity map.
 - Dimensionality Check: The __init__ includes logic to calculate the expected sequence length after the 3D encoder based on the chosen strides and compares it to the sequence length required by the 2D decoder based on its strides and the target output size. This is crucial for the model to be structurally valid.
5. **EMA Utility:** A standard class to manage the Exponential Moving Average of model weights. It creates a shadow copy of the parameters. The update() method is called during training to move the shadow weights towards the current model weights. apply_shadow() copies the smoothed weights into the model (typically for evaluation or inference), and restore() copies the original weights back.

## Architecture 

Okay, let's design a sample PyTorch model that combines a 3D ConvNeXt-like encoder for processing the 4D waveform data (S, T, R) per sample, a Transformer encoder for global feature integration, a decoder to reconstruct the 2D velocity map (H, W), and incorporates EMA for training stability.

The main challenge is mapping the 4D input (batch_size, num_sources, time_steps, num_receivers) to the 2D output (batch_size, height, width). The 3D CNN will process the (num_sources, time_steps, num_receivers) part as a 3D volume (potentially adding a channel dimension first). It will extract features and reduce dimensions. The Transformer will process the flattened feature volume or a sequence derived from it. The decoder will take the Transformer output and reconstruct the 2D spatial grid.

##### Here's a possible architecture flow:

1. **Input:** (B, S, T, R) -> Add channel: (B, 1, S, T, R)
2. **3D ConvNeXt Encoder:** Process (B, 1, S, T, R) through stages of 3D ConvNeXt blocks with downsampling. Output: (B, C_3d, S', T', R').
3. **Flatten & Project to Sequence:** Reshape (B, C_3d, S', T', R') to (B, S'*T'*R', C_3d). Apply a Linear layer to project C_3d to Transformer's embed_dim. Add positional embeddings. Output: (B, SeqLen, embed_dim), where SeqLen = S'*T'*R'.
4. **Transformer Encoder:** Process the sequence (B, SeqLen, embed_dim) through standard Transformer layers. Output: (B, SeqLen, embed_dim).
5. **Map to 2D Latent Grid:** Apply a Linear layer to project embed_dim to decoder's initial channels C_dec_init. Reshape (B, SeqLen, C_dec_init) to (B, C_dec_init, H_latent, W_latent), where H_latent * W_latent = SeqLen. This reshape is a critical assumption about how the sequence maps back to space. We'll need to ensure SeqLen allows for a reasonable H_latent, W_latent.
6. **2D Decoder:** Use 2D Transposed Convolutions (nn.ConvTranspose2d) to upsample from (B, C_dec_init, H_latent, W_latent) to (B, C_final, H, W).
7. **Output Layer:** A final 2D Convolution to get (B, 1, H, W).
And the EMA will be a utility applied externally during training.